## Visa Types
When it comes to traveling and *living abroad* in general, one of the major decisions one must make is related to the [visas](https://en.wikipedia.org/wiki/Travel_visa) used to enter the desired country. Basically all visas can be grouped into *two basic* categories:
+ Long-Term
+ Short-Term

The difference between the two is simply the duration of stay. As you could imagine *long-term* visas entail at least a *year or more* stay, where as *short-term* are at most *3 months*. Thus different *strategies* are needed depending on the *type* of visa you choose.

## Long-Term Visa Strategies
Since the *long-term* visa will typically include *at least a year or more* of duration in the foreign country, we can begin to understand the advantage in terms of the additional months $\Delta{N}$ acquired by residing/operating in the *foreign* country compared to the *country of origin*:

$$
\Delta{N} = N_{foreign} + N_{remainder} - N_{domestic}
$$

Where $N_{foreign}$ is the total number of months that your wealth $W$ can purchase for you under a given *cost of living* $C$, $N_{remainder}$ is the *remaining wealth* (if any) that will be used to live in the *country of origin* upon return from the *foreign* country, and $N_{domestic}$ is simply the numer of months that your wealth $W$ can purchase for you under the *cost of living* $C$ in your home country (never having traveled to the *foreign* country). The full equation is shown below:

$$
\Delta{N} = \min\left(\frac{(1 - p)W}{C_{f}}, N_{limit}\right) + \frac{(1 - p)W - \min\left(\frac{(1 - p)W}{C_{f}}, N_{limit}\right)C_f}{C_d} - \frac{W}{C_{d}}
$$

Where the $C_{f}$ and $C_{d}$ terms are the *cost of living* (including renting, utilities, food, etc ...) for both the *foreign* and *domestic* countries respectively. The value $(1 - p)W$ is the remaining wealth after $pW$ is used to pay various costs related to *travel* $T$, *visa fees* $V$, *investment* $I$, etc ... into the target foreign country:

$$
pW = T + V + I \dots
$$

The term $N_{limit}$ is the *maximum* amount of months the specific visa *allows*, and if the $(1 - p)W$ term can purchase *more months* (with *cost of living* $C_f$) than $N_{limit}$ allows, then the *smaller* of the two values (i.e. $N_{limit}$) will be chosen. That is the purpose of this component of the equation:

$$
\min\left(\frac{(1 - p)W}{C_{f}}, N_{limit}\right)
$$

And of course $p$ is the *fraction of wealth* used to cover the necessary costs (i.e. *travel*, *visa fees*, *investment*, etc ...) for entering and residing in the target *foreign country*.

## Brazil
As an example of the *long-term* visa strategy, let us look at the *average U.S. citizen* circa 2022 (who you [may recall](https://diogenesanalytics.com/blog/2024/05/12/rent-or-buy) has $\sim62\text{k}$ USD in savings) residing $1$, $2$, $4$, and $8$ years in *Brazil*.

In [ ]:
# libs for downloading earth map data
import pathlib
import tempfile
import urllib.request
import zipfile
from tqdm import tqdm

# natural earth 10m cultural shape file
shape_file_url = (
    "https://naciscdn.org/naturalearth/10m/cultural/10m_cultural.zip"
)

# directory to save the downloaded file
data_dir = pathlib.Path("./data/geo/")

# directory to extract the contents of the ZIP file
extract_dir = data_dir / "natural_earth_10m_cultural"

def download_with_progress(url: str, filename: pathlib.Path) -> None:
    """Utility function to download files with progress bar."""
    # set headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # create request
    request = urllib.request.Request(url, headers=headers)

    # open URL
    response = urllib.request.urlopen(request)

    # get total file size
    total = int(response.getheader('Content-Length', 0))

    # set block size for updating
    block_size = 1024

    # create progress bar
    tqdm_bar = tqdm(total=total, unit='B', unit_scale=True, desc=filename.name)

    # open new file
    with open(filename, 'wb') as file:
        # loop over blocks
        while True:
            # get next data chunk
            buffer = response.read(block_size)

            # quit if finished
            if not buffer:
                break

            # write out data chunk
            file.write(buffer)

            # update progress bar
            tqdm_bar.update(len(buffer))

    # finished
    tqdm_bar.close()

# check for previous download
if not extract_dir.exists():
    # create the parent tree (in case it doesn't exist)
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # create a temporary directory to download the file
    with tempfile.TemporaryDirectory() as temp_dir:
        # filename for the downloaded ZIP file
        zip_filename = pathlib.Path(temp_dir) / "10m_cultural.zip"
        
        # download the ZIP file with progress bar
        download_with_progress(shape_file_url, zip_filename)
    
        # extract the contents of the ZIP file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

In [ ]:
# now libs for loading earth map
import geopandas as gpd

# get world geo data
world_map = gpd.read_file(extract_dir / "10m_cultural")

# Filter for South American countries
south_america = world_map[(world_map["CONTINENT"] == "South America")]

In [ ]:
# get libs for downloading
import pandas as pd

# get cost of living data
col_data = pd.read_csv(
    "https://gist.githubusercontent.com/DiogenesAnalytics/ef50827b9b5d8ae5a2617e8a85f0a789/raw/ba7da0101aa98f163e567dc809e4ed0509e5fdb1/col_usd_world_dp_094219_01062024.csv",
    dtype={
        "country": str,     
        "col_usd": int,
    },
)

In [ ]:
# get plotting libs
import matplotlib.pyplot as plt

# set initial figure counter to 1
fig_count = 1

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# get subplot for South America
fig, ax = plt.subplots(figsize=(6, 6))

# set the view for only the mainland
ax.set_xlim([-85, -30])
ax.set_ylim([-60, 15])

# turn off axes for the South America plot only
ax.axis("off")

# set title
plt.suptitle(
    f"Figure {fig_count}. Map of South America (Brazil Highlighted)", y=0.0001, fontsize=10,
)

# adjust padding
plt.tight_layout(pad=0.5)  # Adjust the padding as needed

# increment fig count
fig_count += 1

# plot all countries with darkish grey color
south_america.plot(
    ax=ax,
    color="grey",
    edgecolor="#181818",
    linewidth=0.1,
);

# plot Brazil with a different color
south_america[south_america["ISO_A2"] == "BR"].plot(
    ax=ax,
    color=plt.cm.viridis(0.70),
    edgecolor="#181818",
);

In [ ]:
# get type for sequence
from typing import Sequence, Union

# define functions for calculating additional months
def calculate_delta_n(
    p: float,
    W: float,
    d: Union[Sequence[int], int],
    S: Union[Sequence[int], int],
    C_f: Union[Sequence[float], float],
    C_d: float) -> float:
    """Calculate the additional months gained through immigration."""
    # check for single values and convert to sequences if necessary
    d = d if isinstance(d, Sequence) else [d]
    S = S if isinstance(S, Sequence) else [S]
    C_f = C_f if isinstance(C_f, Sequence) else [C_f]
    
    # ensure lengths of d, S, and C_f are consistent
    assert (len(d) == len(S) == len(C_f)), "The lengths of d, S, and C_f must be the same."
    
    # calculate the days staying such that d_i <= S_i
    max_days_staying = [min(d_i, S_i) for d_i, S_i in zip(d, S)]

    # calculate total days of travel
    total_days_travel = sum(max_days_staying)

    # check total > 0
    assert total_days_travel > 0, "Total days of travel must be greater than zero."
    
    # compute the weighted sum of C_f
    weighted_C_f = sum((max_days_staying[i] / total_days_travel) * c for i, c in enumerate(C_f))

    # calculate wealth remaining after expenses
    remaining_wealth = ((1 - p ) * W)
    
    # calculate the first term inside the min function
    N_foreign = remaining_wealth / weighted_C_f
    
    # calculate the second term inside the min function
    N_limit = sum(S_i / 30.4375 for S_i in S)
    
    # calculate the result of the min function
    N_min = min(N_foreign, N_limit)

    # calculate months for remainder of wealth after travel
    N_remainder = (remaining_wealth - (N_min * weighted_C_f)) / C_d

    # calculate months in home country
    N_domestic = W / C_d
    
    # calculate the final result
    delta_N_value = N_min + N_remainder - N_domestic
    
    return delta_N_value

def absolute_objective_function(*args, **kwargs):
    """Calculate absolute value for objective function."""
    return abs(calculate_delta_n(*args, **kwargs))

In [ ]:
# get optimization libs
import numpy as np
from scipy.optimize import minimize

# set values
average_US_savings = 62000
average_US_col = col_data[col_data["country"] == "United States"]["col_usd"].values[0]
average_BR_col = col_data[col_data["country"] == "Brazil"]["col_usd"].values[0]

# generate p values
p_values = np.arange(0, 1 + 0.01, 0.01)

# loop over various visa lengths
for i, guess in zip(range(0, 4), (0.25, 0.55, 0.7, 0.7)):
    # calculate visa in days
    visa_days = (2 ** i) * 365

    # get the delta_N values
    delta_n_values = np.array(
        [
            calculate_delta_n(p, average_US_savings, visa_days, visa_days, average_BR_col, average_US_col)
            for p in p_values
        ]
    )
    
    # plotting the data
    plt.plot(p_values, delta_n_values, label=f"{2 ** i} year")

    # find the value of p that minimizes the absolute value of the objective function
    result = minimize(
        absolute_objective_function,
        guess,
        args=(average_US_savings, visa_days, visa_days, average_BR_col, average_US_col),
    )
    
    # extract the optimal point as x/y pair
    threshold_x = result.x[0]
    threshold_y = calculate_delta_n(
        threshold_x, 
        average_US_savings,
        visa_days,
        visa_days,
        average_BR_col,
        average_US_col
    )

    # plot threshold point
    plt.plot(
        threshold_x, 
        threshold_y,
        marker="o",
        markersize=7,
        color="green"
    )

    # plot threshold vertical line
    threshold_line = plt.axvline(x=threshold_x, linestyle='--', color="green")

# adding labels and title
plt.xlabel("% Wealth (p)")
plt.ylabel("Additional Months (ΔN)")

# add x-axis
plt.axhline(y=0, linestyle='--', color="black", label="x-axis")

# label thresholds
threshold_line.set_label("Advantage Threshold")

# set title
plt.suptitle(
    f"Figure {fig_count}. Average U.S. Citizen in Brazil with Long-Term Visa", y=0.0001, fontsize=10
)

# increment fig count
fig_count += 1

# displaying the plot
plt.legend()
plt.show()

*Figure 2* shows some interesting results. Naturally it makes sense that as the *visa duration* $N_{limit}$ increases from $1$ to $8$ years, the *additional months* $\Delta{N}$ gained increases (because you are not *limited* by the *visa's duration* and can buy more time at a lower $C_f$ in the *foreign country*). But what is the nature of the strange *"angle"* or *"kink"* in each line? As it turns out, everything *preceeding* this *"kink"* is the result of the *remaining wealth* $(1 - p)W$ **exceeding** $N_{limit}$:

$$
(1 - p)W > N_{limit}
$$

Naturally, everything to the right of this *"kink"* is the result of the remaining wealth being **less** than $N_{limit}$:

$$
(1 - p)W < N_{limit}
$$

As $N_{limit}$ increases from $1 - 8$ years, we see the kink shift to the left. Why? Again, because more and more of your *remaining wealth* $(1 - p)W$ can be used to buy *more months* $N$.


Another *key insight* worth pointing out is that everything *below* the x-axis (i.e. when $\Delta{N} \lt 0$) is no longer *favorable*. Why? Because the `% wealth` $pW$ you set aside for *travel* $T$, *visa* $V$, *investment* $I$, and any other costs, leaves a *remaining wealth* $(1 - p)W$ that buys *less months* $N$ than if you stayed in your *home country* (in this case the *U.S.*). To put it more clearly:

$$
N_{foreign} + N_{remainder} < N_{domestic}
$$

Finally, why does the *"break even"* point shift to the right as the *visa duration* increases from $1$, $2$, $4$, and $8$? Keep in mind that the equation accounts for the $N_{remainder}$ which is the amount of months that can be purchased with any wealth remaining after *leaving* the foreign country and returning home. So what we are seeing is, that as the *visa duration* increases, and you are legally allowed to reside in the *foreign country* for longer time, you can purchase *the same amount* of months for a *smaller percent* of your wealth. Or to put it another way, the fraction of wealth $p$ set aside for the various costs ($T$, $V$, $I$, etc ...), can increase:

$$
\begin{align*}
    \Delta{N}_8 &= \Delta{N}_4 = \Delta{N}_2 = \Delta{N}_1 = N_{foreign} + N_{remainder} - N_{domestic} = 0 \\
    (1 - p_8) &= (1 - p_4) < (1 - p_2) < (1 - p_1) \\
    p_8 &= p_4 > p_2 > p_1
\end{align*}
$$

Notice that the *"break even"* point for both the $4$ and $8$ year visa is the same. Why? Because they are *both under* their max visa duration:

$$
\frac{(1 - p)W}{C_f} < N_{limit}
$$

And when this happens the equation simplifies to:

$$
\begin{align*}
    \frac{(1 - p)W}{C_{f}} + \frac{(1 - p)W - \frac{(1 - p)W}{C_{f}}C_f}{C_d} &- \frac{W}{C_{d}} \\
    \frac{(1 - p)W}{C_{f}} + \frac{0}{C_d} &- \frac{W}{C_{d}} \\
    \frac{(1 - p)W}{C_{f}} &- \frac{W}{C_{d}}
\end{align*}
$$

With the visa duration $N_{limit}$ no longer in the equation, the results of the $4$ and $8$ year visas are the same.

All of this is to state a very simple fact: the more time you can spend under the *lower cost of living* $C_f$ the more wealth $pW$ you can use on any and all costs, and the less *remaining wealth* $(1 - p)W$ you need to maintain the minimum months of wealth (i.e. $\Delta{N} = N_{domestic}$).

## Short-Term Visa Strategies
The *short-term* visas are usually measured in *days* (e.g. $30$, $60$, $90$), and the main difference between the two *strategies* is that for the short-term strategy we want to *combine* multiple countries (their $C_f$ and $T$, $V$, $I$, costs) together to take advantage of possible *advantageous combinations*:

$$
\begin{align*}
    N_{limit} &= \sum_{i=1}^n \frac{S_i}{30.4375} \\
    D_{total} &= \sum_{i=1}^n \min\left(d_i, S_i\right) \\
    C_f &= \sum_{i=1}^n \frac{\min\left(d_i, S_i\right)}{D_{total}}C_i \\
    pW &= \sum_{i=1}^n T_i + V_i + I_i
\end{align*}
$$

Here $S_i$ is the *max visa days*, $d_i$ is the *days chosen* to stay in the country such that $d_i <= S_i$, $D_{total}$ is the total days of residing in *all countries*, and $T_i$, $V_i$, and $I_i$ are the necessary costs, all for the *i-th* country out of $n$ total countries that will be visited for $d_i$ days.

## The South America Tour
Let us now apply the *short-term* strategy to South America, where instead of traveling *only* to Brazil we will stay in multiple countries for the *max visa days* $S_i$ in each.

In [ ]:
# create map from country name to ISO2 name
south_american_countries_iso2 = {
    "Argentina": "AR", 
    "Bolivia": "BO", 
    "Brazil": "BR", 
    "Chile": "CL", 
    "Colombia": "CO", 
    "Ecuador": "EC", 
    "Guyana": "GY", 
    "Paraguay": "PY", 
    "Peru": "PE", 
    "Suriname": "SR", 
    "Uruguay": "UY", 
    "Venezuela": "VE"
}

# get passport data
passport_data = (
    pd.read_csv(
    "https://gist.githubusercontent.com/DiogenesAnalytics/c0b0cb5b6269b234302ad6bf8810531c/raw/6d909afc842e3bf039f7f1bd3064595b3642e4d4/passport-index-matrix-iso2_522919_31052024.csv",
    dtype=str
    )
    .fillna("NA")
)

In [ ]:
# create pointer to US passport holder
us_passport = passport_data[passport_data["Passport"] == "US"]

# filter sa countries that do not have their days listed
target_sa_countries = {
    country: code
    for country, code in south_american_countries_iso2.items()
    if "visa" not in us_passport[code].values[0]
}

# creating the C_f and S sequences for all south american countries
C_f_sa_trip, S_sa_trip = zip(*(
    (col_data[col_data["country"] == country]["col_usd"].values[0], int(us_passport[code].values[0]))
    for country, code in target_sa_countries.items()
))

In [ ]:
# get total days from max visa stay
sa_trip_total_days = sum(S_sa_trip)

# now create list of weighted C_f values
weighted_C_f_values = {
    value: c * (s / sa_trip_total_days)
    for (c, s, value) in zip(C_f_sa_trip, S_sa_trip, target_sa_countries.values())
}

# create new map of only target countries
sa_tour_map = south_america[south_america["ISO_A2"].isin(weighted_C_f_values.keys())].copy()

# add cost of living data
sa_tour_map["COL"] = sa_tour_map["ISO_A2"].map(weighted_C_f_values)

In [ ]:
# get subplot for South America
fig, ax = plt.subplots(figsize=(6, 6))

# set the view for only the mainland
ax.set_xlim([-85, -30])
ax.set_ylim([-60, 15])

# turn off axes for the South America plot only
ax.axis("off")

# set title
plt.suptitle(
    f"Figure {fig_count}. Cost of Living Choropleth Map of South America Tour", y=0.0001, fontsize=10,
)

# adjust padding
plt.tight_layout(pad=0.5)  

# increment fig count
fig_count += 1

# plot all countries with darkish grey color
south_america.plot(
    ax=ax,
    color="grey",
    edgecolor="#181818",
    linewidth=0.1,
);

# plot Brazil with a disfferent color
sa_tour_map.plot(
    ax=ax,
    cmap="viridis",
    column="COL",
    edgecolor="#181818",
    legend=True,
    legend_kwds={"label": "Weighted Cost of Living (USD)", "orientation": "vertical"},
);

In [ ]:
# find the value of p that minimizes the absolute value of the objective function
result = minimize(
    absolute_objective_function,
    0.5,
    args=(average_US_savings, S_sa_trip, S_sa_trip, C_f_sa_trip, average_US_col),
)

# Extract the optimal value of x
threshold_p = result.x[0]

# get the delta_N values
delta_n_values = np.array(
    [
        calculate_delta_n(p, average_US_savings, S_sa_trip, S_sa_trip, C_f_sa_trip, average_US_col)
        for p in p_values
    ]
)

# plotting the data
plt.plot(p_values, delta_n_values)

# plot threshold p value
plt.axvline(x=threshold_p, color='green', linestyle='--', label='Advantage Threshold')

# adding labels and title
plt.xlabel("% Wealth (p)")
plt.ylabel("Additional Months (ΔN)")

# set title
plt.suptitle(
    f"Figure {fig_count}. Average U.S. Citizen in South America with Short-Term Visa", y=0.0001, fontsize=10
)

# increment fig count
fig_count += 1

# displaying the plot
plt.legend()
plt.show()

*Figure 3* gives us a sense of the *weight* of each country's *cost of living* for this trip (where the *lighter* the color the more weight that country's cost of living has in the $C_f$ value). Again this is a result of the way we calculate $C_f$:

$$
C_f = \sum_{i=1}^n \frac{\min\left(d_i, S_i\right)}{D_{total}}C_i
$$

*Figure 4* shows us that, *worse case scenario*, the *average U.S. citizen* could spend $60\%$ (i.e. $0.60W$) of their wealth (in this case the *average U.S. savings*) on this tour, and you would have the same amount of *wealth time* $N$ as you would had you stayed in the *U.S.*:

$$
N_{foreign} + N_{remainder} = N_{domestic}
$$

This is quite an interesting result, as it also means the *average U.S. citizen* only needs $40\%$ of their wealth to maintain the same amount of wealth time (in months $N$) as you would in the *domestic country*:

$$
\frac{0.40W}{C_f} = \frac{W}{C_d}
$$

## Moral
Once a method for calculating the *advantage* (in this case the $\Delta{N}$ value) is defined, it becomes possible to see the truth about whatever may be the *target* of the calculation. Here, we have been able to investigate the *additional months* $\Delta{N}$ gained by partitioning your wealth $W$ into a *travel* partition $pW$ and a *cost of living* partition $(1 - p)W$. What we can confidently state, is that there are combinations of $N_{limit}$, and $p$, that lead to significantly *favorable* outcomes (e.g. $\sim 3$ x increase with $\frac{N_{foreign}}{N_{domestic}}$ in the case of the *Brazil long-term* $8$ year visa). Further investigations into possible *short-term* strategies, whereby certain countries are selected due to their *low cost of living* $C_f$ and their relative proximity to each other, may hold significant *potential advantage*.